# Interpolation based on Radial Basis Functions

Parameter Matrix (2D meshgrid)

In [1]:
P=[1 2 3 1 2 3 1 2 3;1 1 1 2 2 2 3 3 3]

2×9 Array{Int64,2}:
 1  2  3  1  2  3  1  2  3
 1  1  1  2  2  2  3  3  3

2D coordinates of the 3 points to be interpolated

In [2]:
 Pint=[1.5 1.1 2.5;1.5 1.2 2.5]

2×3 Array{Float64,2}:
 1.5  1.1  2.5
 1.5  1.2  2.5

# Function used to generate the response for the previous 9 datapoints 

In [3]:
f(x)=[sqrt(x[1]), x[2]^2, x[1]+x[2]]

f (generic function with 1 method)

# Generation of the data matrix

In [4]:
Y=zeros(3,9)
for i in 1:9
    Y[:,i]=f(P[:,i])
end
println("\rThe Data - Snapshot matrix Y coming out from the DOE\r")
round.(Y;digits=2)

The Data - Snapshot matrix Y coming out from the DOE


3×9 Array{Float64,2}:
 1.0  1.41  1.73  1.0  1.41  1.73  1.0  1.41  1.73
 1.0  1.0   1.0   4.0  4.0   4.0   9.0  9.0   9.0
 2.0  3.0   4.0   3.0  4.0   5.0   4.0  5.0   6.0

# Choice of two types of Radial Basis Function :
## 1. Linear Spline (lin)
## 2. Cubic Spline (cub)

1. RBF : Linear Spline

In [5]:
function GlinMtx(p)
    M = size(p, 1)
    N = size(p, 2)
    G = zeros(N,N)
    for i = 1:N
        for j = 1:N
            G[i,j] = sqrt(sum((p[:,i] - p[:,j]).^2)) # G1[i,j]=sqrt((P[:,i][k]-P[:,j][1])^2+(P[:,i][k]-P[:,j][2])^2) for k={1:2}
        end
    end
    return G
end

GlinMtx (generic function with 1 method)

In [6]:
Glin=GlinMtx(P)

9×9 Array{Float64,2}:
 0.0      1.0      2.0      1.0      …  2.23607  2.0      2.23607  2.82843
 1.0      0.0      1.0      1.41421     1.41421  2.23607  2.0      2.23607
 2.0      1.0      0.0      2.23607     1.0      2.82843  2.23607  2.0
 1.0      1.41421  2.23607  0.0         2.0      1.0      1.41421  2.23607
 1.41421  1.0      1.41421  1.0         1.0      1.41421  1.0      1.41421
 2.23607  1.41421  1.0      2.0      …  0.0      2.23607  1.41421  1.0
 2.0      2.23607  2.82843  1.0         2.23607  0.0      1.0      2.0
 2.23607  2.0      2.23607  1.41421     1.41421  1.0      0.0      1.0
 2.82843  2.23607  2.0      2.23607     1.0      2.0      1.0      0.0

Calculated Matrix A of coefficients of interpolation 

In [7]:
Alin=Y*(inv(Glin'))'

3×9 Array{Float64,2}:
 0.367804  -0.0436572  0.152806  …   0.367804  -0.0436572   0.152806
 1.72823    0.161643   1.72823      -0.621314  -0.934163   -0.621314
 1.30171    0.126765   0.714327      0.714327  -0.147186    0.126941

2. RBF : Cubic Spline

In [8]:
function GcubMtx(p)
    M = size(p, 1)
    N = size(p, 2)
    G = zeros(N,N)
    for i = 1:N
        for j = 1:N
            G[i,j] = sqrt(sum((p[:,i] - p[:,j]).^2))^3 # G1[i,j]=sqrt((P[:,i][k]-P[:,j][1])^2+(P[:,i][k]-P[:,j][2])^2)^3 for k={1:2}
        end
    end
    return G
end

GcubMtx (generic function with 1 method)

In [9]:
Gcub=GcubMtx(P)

9×9 Array{Float64,2}:
  0.0       1.0       8.0       1.0      …   8.0      11.1803   22.6274
  1.0       0.0       1.0       2.82843     11.1803    8.0      11.1803
  8.0       1.0       0.0      11.1803      22.6274   11.1803    8.0
  1.0       2.82843  11.1803    0.0          1.0       2.82843  11.1803
  2.82843   1.0       2.82843   1.0          2.82843   1.0       2.82843
 11.1803    2.82843   1.0       8.0      …  11.1803    2.82843   1.0
  8.0      11.1803   22.6274    1.0          0.0       1.0       8.0
 11.1803    8.0      11.1803    2.82843      1.0       0.0       1.0
 22.6274   11.1803    8.0      11.1803       8.0       1.0       0.0

Calculated Matrix A of coefficients of interpolation 

In [10]:
Acub=Y*(inv(Gcub'))'

3×9 Array{Float64,2}:
 0.27631   -0.331726  0.215527  -0.383887  …  0.27631   -0.331726  0.215527
 0.980982  -0.967142  0.980982  -1.04863      0.316732  -0.276571  0.316732
 0.856022  -1.03782   0.68996   -1.03782      0.68996   -0.865181  0.523898

## Calculate the RBF interpolated values of the points Pint to be interpolated  

1. RBF : Linear Spline

In [22]:
g=zeros(9,3);

In [ ]:
#Wrong
for j in 1:3
    for i=1:9
    g[i,j]=sqrt((Pint[:,j][1]-P[:,j][1])^2+(Pint[:,j][2]-P[:,j][2])^2)
end
end
g

In [15]:
#Node 1 (1.5,1.5)
g₁¹=zeros(9)
for i in 1:9
g₁¹[i]=sqrt((1.5-P[:,i][1])^2+(1.5-P[:,i][2])^2)
end
g₁¹

9-element Array{Float64,1}:
 0.7071067811865476
 0.7071067811865476
 1.5811388300841898
 0.7071067811865476
 0.7071067811865476
 1.5811388300841898
 1.5811388300841898
 1.5811388300841898
 2.1213203435596424

In [16]:
#Node 2 (1.1,1.1)
g₁²=zeros(9)
for i in 1:9
g₁²[i]=sqrt((1.1-P[:,i][1])^2+(1.2-P[:,i][2])^2)
end
g₁²

9-element Array{Float64,1}:
 0.22360679774997896
 0.9219544457292886
 1.91049731745428
 0.806225774829855
 1.2041594578792296
 2.0615528128088303
 1.8027756377319948
 2.0124611797498106
 2.6172504656604803

In [17]:
#Node 3 (2.5,2.5)
g₁³=zeros(9)
for i in 1:9
g₁³[i]=sqrt((2.5-P[:,i][1])^2+(2.5-P[:,i][2])^2)
end
g₁³

9-element Array{Float64,1}:
 2.1213203435596424
 1.5811388300841898
 1.5811388300841898
 1.5811388300841898
 0.7071067811865476
 0.7071067811865476
 1.5811388300841898
 0.7071067811865476
 0.7071067811865476

In [19]:
f₁¹=Alin*g₁¹

3-element Array{Float64,1}:
 1.138303850496928
 2.0769803445205515
 2.756639112959866

In [20]:
f₁²=Alin*g₁²

3-element Array{Float64,1}:
 1.0177206443731015
 1.4070346853046365
 2.204142798417518

In [21]:
f₁³=Alin*g₁³

3-element Array{Float64,1}:
 1.5299989984668725
 6.357504525848947
 4.896901203624063

2. RBF : Cubic Spline

In [23]:
#Node 1 (1.5,1.5)
g₂¹=zeros(9)
for i in 1:9
g₂¹[i]=sqrt((1.5-P[:,i][1])^2+(1.5-P[:,i][2])^2)^3
end
g₂¹

9-element Array{Float64,1}:
 0.35355339059327384
 0.35355339059327384
 3.952847075210475
 0.35355339059327384
 0.35355339059327384
 3.952847075210475
 3.952847075210475
 3.952847075210475
 9.545941546018389

In [24]:
#Node 2 (1.1,1.1)
g₂²=zeros(9)
for i in 1:9
g₂²[i]=sqrt((1.1-P[:,i][1])^2+(1.2-P[:,i][2])^2)^3
end
g₂²

9-element Array{Float64,1}:
  0.011180339887498947
  0.7836612788698952
  6.973315208708123
  0.5240467536394059
  1.7460312139248833
  8.761599454437528
  5.859020822628984
  8.150467777986734
 17.928165689774293

In [25]:
#Node 3 (2.5,2.5)
g₂³=zeros(9)
for i in 1:9
g₂³[i]=sqrt((2.5-P[:,i][1])^2+(2.5-P[:,i][2])^2)^3
end
g₂³

9-element Array{Float64,1}:
 9.545941546018389
 3.952847075210475
 3.952847075210475
 3.952847075210475
 0.35355339059327384
 0.35355339059327384
 3.952847075210475
 0.35355339059327384
 0.35355339059327384

In [26]:
f₂¹=Acub*g₂¹

3-element Array{Float64,1}:
 1.2921213776313536
 2.584422378781933
 3.2506384352761466

In [27]:
f₂²=Acub*g₂²

3-element Array{Float64,1}:
 1.1367423567255974
 1.811487244689924
 2.5925893416474644

In [28]:
f₂³=Acub*g₂³

3-element Array{Float64,1}:
 1.6234173643382126
 6.204892250889171
 5.060873371329757

## Calculated Interpolations for the 3 points in Pint using a linear spline

In [29]:
f₁=[f₁¹ f₁² f₁³]

3×3 Array{Float64,2}:
 1.1383   1.01772  1.53
 2.07698  1.40703  6.3575
 2.75664  2.20414  4.8969

## Calculated Interpolations for the 3 points in Pint using a cubic spline

In [30]:
f₂=[f₂¹ f₂² f₂³]

3×3 Array{Float64,2}:
 1.29212  1.13674  1.62342
 2.58442  1.81149  6.20489
 3.25064  2.59259  5.06087

## To be compared to the exact solution

In [31]:
fsol=[f([1.5,1.5]) f([1.1,1.2]) f([2.5,2.5])]

3×3 Array{Float64,2}:
 1.22474  1.04881  1.58114
 2.25     1.44     6.25
 3.0      2.3      5.0

## Errors

In [ ]:
?sum